# **Bing Image Creator DALL-E 3 Batch v3**
- For more details: [GitHub](https://github.com/meap158/bing_image_creator_dalle3_batch)
- Feel free to [PM me on my Facebook profile](https://www.facebook.com/messages/t/meap9999) for support.

---

### **How to Obtain Bing Cookie**
To use Bing Image Creator, you need to obtain the necessary cookie value. Follow these steps:

1. Navigate to Bing Image Creator: https://www.bing.com/images/create.
2. Log in to your Bing account.
3. Open the browser's developer tools:
   - Press `F12` or right-click on the page and select "Inspect" or "Inspect Element."
4. In the Developer Tools, go to the "Application" tab.
5. Under "Cookies," find and select `https://www.bing.com`.
6. Look for the `_U` cookie.
7. Copy the value of the `_U` cookie.

---
### **Okay now I have the `_U` cookie, where do I put it?**
#### *Option 1. For single cookie use:*
1. Just input your `_U` cookie value into the `bing_cookie` parameter.

#### *Option 2. For multi-cookie use (i.e., multiple accounts):*

1. Create a Secret named `bing_cookies_yaml` in the Secrets tab (the one with the key icon):
2. Input your `_U` cookie value(s) into the Value column following this syntax (the names on the left side don't matter much as they're just for readability. You could just put the names of your accounts there):
    ```
    account1: {_U cookie value}

    account2: {_U cookie value}
    ```
3. Enable Notebook access to the Secret.

---
### **How to Use**
1. Fill in the necessary parameters: `spreadsheet_id_or_url`, `worksheet_name_or_index`, and `range_value`, as per your Google Sheets setup. The values from this specified range will be fetched and used as prompts.
2. Press `Ctrl + F9` to run all cells.

In [ ]:
# @title Set up class GoogleDrive {display-mode: "form"}

from google.colab import drive
import os
import shutil

class GoogleDrive:
    def __init__(self, mount_folder='/content/drive'):
        self.mount_folder = mount_folder
        if not self.is_drive_mounted():
            drive.mount(self.mount_folder)

    def is_drive_mounted(self):
        # Check if the mount folder exists
        return os.path.exists(self.mount_folder)

    def create_folder(self, folder_name, parent_folder='MyDrive'):
        full_parent_path = os.path.join(self.mount_folder, parent_folder)
        folder_path = os.path.join(full_parent_path, folder_name)
        # Create both the output folder and the step folder inside it
        os.makedirs(folder_path, exist_ok=True)
        return folder_path

gdrive_client = GoogleDrive()

Mounted at /content/drive


In [ ]:
# @title Prompt auth beforehand for class GoogleSheets {display-mode: "form"}

from google.colab import auth
auth.authenticate_user()

In [ ]:
# @title Install Dependencies {display-mode: "form"}

%%capture

# Update and install chromium-chromedriver
!apt-get update
!apt-get install chromium-chromedriver -y
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# Install Python packages
!pip install requests
!pip install selenium
!pip install seleniumbase
!pip install piexif
!pip install gspread
!pip install PyYAML

# Set up Utils and Classes

In [ ]:
# @title Utils {display-mode: "form"}
import re

def remove_text_in_brackets(text):
    # Remove all text within brackets (including the brackets themselves).
    return re.sub(r'<[^>]+>', '', text)

def print_colored_text(text, color, bold=True):
    color_mapping = {
        'black': '\033[30m',
        'blue': '\033[34m',
        'red': '\033[31m',
        'yellow': '\033[33m',
        'green': '\033[32m',
    }
    color_code = color_mapping.get(color.lower(), '')
    # Check if bold is requested
    if bold:
        color_code += '\033[1m'
    reset_code = '\033[0m'
    print(color_code + text + reset_code)

In [ ]:
# @title Set up class BingImageCreator {display-mode: "form"}
import os
import random
import string
import time
import requests
# from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from seleniumbase import Driver
from urllib.parse import urlparse, urlunparse

from IPython.display import Image, HTML
import piexif
from PIL import Image as PILImage

class BingImageCreator:
    def __init__(self, prompts, cookie_value=None, save_folder="images"):
        self.prompts = prompts if isinstance(prompts, list) else [prompts]
        self.cookie_value = cookie_value
        self.save_folder = save_folder
        self.current_prompt = ""
        # Setup driver
        self.driver = Driver(headless2=True, no_sandbox=True)

    def set_prompts(self, new_prompts):
        self.prompts = new_prompts if isinstance(new_prompts, list) else [new_prompts]

    def set_save_folder(self, new_save_folder):
        self.save_folder = new_save_folder

    def navigate_to_bing_create_page(self):
        self.driver.get("https://www.bing.com/images/create")

    def set_cookie(self):
        cookie = {"name": "_U", "value": self.cookie_value}
        self.driver.add_cookie(cookie)

    def enter_prompt(self, prompt):
        try:
            # Find the search box by its ID and enter a new prompt
            search_box = self.driver.find_element("id", "sb_form_q")
            search_box.clear()  # Clear existing text
            formatted_prompt = prompt.replace("\n", ",").strip()  # Replace newlines with commas and strip whitespace
            limited_prompt = formatted_prompt[:480]  # Limit prompt to 480 characters
            search_box.send_keys(limited_prompt)
            time.sleep(3)
            search_box.send_keys(Keys.RETURN)  # Press Enter
            time.sleep(1)  # Introduce a small delay (1 second) for stability
        except Exception as e:
            print(f"Error: {e}")
            self.capture_and_display_screenshot()

    def wait_for_loader(self):
        time.sleep(3)
        try:
            # Check if the prompt has been blocked (presence of class "block_icon")
            if self.driver.find_elements(By.CLASS_NAME, "block_icon"):
                # Print the error messages
                error_mt_elements = self.driver.find_elements(By.CLASS_NAME, "gil_err_mt")
                error_sbt_elements = self.driver.find_elements(By.CLASS_NAME, "gil_err_sbt1")
                if error_mt_elements:
                    print("Error Message (gil_err_mt):", error_mt_elements[0].text)
                if error_sbt_elements:
                    print("Error Message (gil_err_sbt1):", error_sbt_elements[0].text)
                return False
            # Default wait time
            time_to_wait_seconds = 30  # Adjust the timeout as needed
            wait = WebDriverWait(self.driver, time_to_wait_seconds)
            # Wait until the element with ID "giloader" is present
            wait.until(EC.presence_of_element_located((By.ID, "giloader")))  # self.driver.find_element(By.ID, "giloader")
            # Check if boosts have run out (data-tb attribute of the element with id "reward_c" is 0)
            reward_element = self.driver.find_element(By.ID, "reward_c")
            data_tb_value = reward_element.get_attribute("data-tb")
            if data_tb_value == "0":
                # If data-tb is 0, get wait time from the text of the span with id "gi_rmtime" e.g. "1 min wait", "45 sec wait"
                time.sleep(1)
                # Find all elements with ID "gi_rmtime"
                wait_time_elements = self.driver.find_elements(By.ID, "gi_rmtime")
                if wait_time_elements:
                    # If at least one element is found, get wait time from the text of the first element
                    wait_time_text = wait_time_elements[0].text
                    # Split the wait time text into words
                    wait_time_words = wait_time_text.split()
                    # Extract the numerical value and unit from the wait time text
                    value = int(wait_time_words[0])
                    unit = wait_time_words[1]
                    # Convert minutes or seconds to seconds and add it to the wait time
                    if unit == "min":
                        time_to_wait_seconds += value * 60
                    elif unit == "sec":
                        time_to_wait_seconds += value
                else:
                    # If no elements are found, default to 1 minute wait
                    time_to_wait_seconds += 60
            # Wait until the style becomes "display: none;"
            wait = WebDriverWait(self.driver, time_to_wait_seconds)
            print("time_to_wait_seconds:", time_to_wait_seconds)
            wait.until(EC.invisibility_of_element_located((By.ID, "giloader")))
            time.sleep(3)
            return True
        except Exception as e:
            print(f"Error: {e}")
            self.capture_and_display_screenshot()
            return False

    @staticmethod
    def remove_middle_text(text, max_length, indicator="___"):
        """
        Remove excess characters from the middle of the text if its length exceeds the specified maximum.
        """
        if len(text) > max_length:
            excess_chars = len(text) - max_length
            chars_to_remove_from_middle = (
                excess_chars + 1
            ) // 2  # Calculate the number of characters to remove from the middle
            start_index = (len(text) - max_length - chars_to_remove_from_middle) // 2
            end_index = start_index + max_length
            text = text[:start_index] + indicator + text[end_index:]
        return text

    @classmethod
    def sanitize_filename(self, filename):
        """
        Sanitize the given filename to make it compatible across different operating systems.
        """
        valid_chars = set("-_.() %s%s" % (string.ascii_letters, string.digits))
        sanitized_filename = "".join(c if c in valid_chars else "_" for c in filename)
        # Limit the filename length to 196 characters
        max_length = 196
        sanitized_filename = self.remove_middle_text(sanitized_filename, max_length)
        # Remove leading and trailing underscores
        sanitized_filename = sanitized_filename.strip("_")
        # Truncate the filename from the right to a maximum length of 196 characters
        sanitized_filename = (
            sanitized_filename[-max_length:]
            if len(sanitized_filename) > max_length
            else sanitized_filename
        )
        # If the filename is empty, generate a random string and append to "unnamed"
        if not sanitized_filename:
            random_string = "".join(
                random.choice(string.ascii_letters + string.digits) for _ in range(8)
            )
            sanitized_filename = f"unnamed_{random_string}"
        return sanitized_filename

    @staticmethod
    def generate_random_string():
        random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=3))
        return random_string

    def download_images(self):
        try:
            # Select .mimg elements that are descendants of .img_cont
            image_elements = self.driver.find_elements(
                By.CSS_SELECTOR, ".img_cont .mimg"
            )
            image_urls = [element.get_attribute("src") for element in image_elements]
            # Create the save folder if it doesn't exist
            os.makedirs(self.save_folder, exist_ok=True)
            # Download images using alt values as sanitized file names
            random_string = self.generate_random_string()
            for url, element in zip(image_urls, image_elements):
                alt = element.get_attribute("alt")
                sanitized_alt = self.sanitize_filename(alt)
                # Remove unwanted parameters from the URL
                parsed_url = urlparse(url)
                new_url = urlunparse(
                    (
                        parsed_url.scheme,
                        parsed_url.netloc,
                        parsed_url.path,
                        parsed_url.params,
                        "",
                        "",
                    )
                )
                new_url += "?pid=ImgGn"
                # Retrieve the full image content
                response = requests.get(new_url)
                # Modify the file path to save the images to Google Drive
                file_path = os.path.join(self.save_folder, f"{sanitized_alt}_{random_string}.jpg")
                # Write the image content to the file
                with open(file_path, "wb") as f:
                    f.write(response.content)
                print_colored_text(f'Image saved to "{file_path}".', 'blue')
                # Write self.prompt as description metadata of the image
                self.add_description_to_image(file_path, self.current_prompt)

            # Save self.prompt as a text file with the same name as the first sanitized_alt
            first_sanitized_alt = self.sanitize_filename(image_elements[0].get_attribute("alt"))
            prompt_file_path = os.path.join(self.save_folder, "prompts", f"{first_sanitized_alt}.txt")
            os.makedirs(os.path.dirname(prompt_file_path), exist_ok=True)
            with open(prompt_file_path, "w", encoding="utf-8") as prompt_file:
                prompt_file.write(self.current_prompt)
            print_colored_text(f'Prompt saved to "{prompt_file_path}".', 'green')
            # Display preview images in a grid
            self.display_images_in_grid(image_urls)

        except Exception as e:
            print(f"Error: {e}")

    @staticmethod
    def display_images_in_grid(image_urls):
        """
        Display a list of images in a grid layout.
        Args:
            image_urls (list): A list of URLs of the images to display.
        """
        # HTML code for displaying images in a grid
        html_code = "<div style='display: flex; flex-wrap: wrap;'>"
        for url in image_urls:
            html_code += f"<img src='{url}' style='width: 200px; height: 200px; margin: 5px;'>"
        html_code += "</div>"
        # Display the HTML code
        display(HTML(html_code))

    def refresh_and_recheck(self, timeout=10):
        # 0. Check if the "Your images are on the way" popup exists
        try:
            elements = self.driver.find_elements(By.CSS_SELECTOR, 'div.gi_nb.gi_nb_r.show_n')
            if elements:
                popup_text = elements[0].text.replace("\n", ". ")
                print_colored_text(f"Popup found: {popup_text}", "yellow")
                timeout = 150  # Change the timeout if the popup is found
        except:
            pass
        print("Refreshing and rechecking for the loader.")
        self.driver.refresh()
        time.sleep(3)
        # 1. Check if not class="girr_blocked"
        try:
            WebDriverWait(self.driver, timeout).until_not(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#girrcc > div:first-child.girr_blocked'))
            )
            print("The first child div does not have class 'girr_blocked'")
        except Exception as e:
            print(f"The first child div has class 'girr_blocked': {e}")
            return False
        # 2. Check if not imgcount="0"
        unwanted_element = '.girr_set.seled[data-imgcount="0"]'
        try:
            WebDriverWait(self.driver, timeout).until_not(
                EC.presence_of_element_located((By.CSS_SELECTOR, unwanted_element))
            )
            print("The image count is greater than 0.")
            return True
        # Handle timeout by refreshing the page and trying again
        except TimeoutException:
            print("Timeout exceeded. Refreshing the page and trying again.")
            self.driver.refresh()
            try:
                # Wait again after refresh
                WebDriverWait(self.driver, timeout).until_not(
                    EC.presence_of_element_located((By.CSS_SELECTOR, unwanted_element))
                )
                print("The image count is greater than 0 after refresh.")
                return True
            # Handle timeout after refresh
            except TimeoutException:
                print("Timeout exceeded after refresh. The image count is still 0.")
                return False
        # Handle other exceptions
        except Exception as e:
            print(f"Error: {e}")
            # self.capture_and_display_screenshot()
            return False

    def capture_and_display_screenshot(self, output_path='screenshot.png'):
        try:
            self.driver.save_screenshot(output_path)
            display(Image(output_path))
        except Exception as e:
            print(f"Error: {e}")

    @staticmethod
    def add_description_to_image(file_path, description):
        try:
            # Open the image
            img = PILImage.open(file_path)
            # Get the existing Exif data (if any)
            exif_dict = piexif.load(img.info["exif"])
            # Modify or add the 'Description' tag in the Exif data
            exif_dict['0th'][piexif.ImageIFD.ImageDescription] = description
            # Dump the modified Exif data back to bytes
            exif_bytes = piexif.dump(exif_dict)
            # Save the image with the updated Exif data
            img.save(file_path, exif=exif_bytes)
            # Close the image
            img.close()
        except Exception as e:
            print(f"An error occurred while saving description to image: {str(e)}")

    def run(self):
        try:
            for prompt in self.prompts:
                self.navigate_to_bing_create_page()
                self.set_cookie()
                self.driver.refresh()
                # self.capture_and_display_screenshot()
                self.current_prompt = prompt
                self.enter_prompt(self.current_prompt)
                # self.capture_and_display_screenshot()
                if self.wait_for_loader():
                    self.download_images()
                else:
                    if self.refresh_and_recheck():
                        self.download_images()
            self.driver.quit()
        except Exception as e:
            print(f"Error: {e}")

In [ ]:
# @title Set up class GoogleSheets {display-mode: "form"}
import gspread
from google.auth import default
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError

class GoogleSheets:
    def __init__(self, spreadsheet_id_or_url, worksheet_name_or_index):
            self.spreadsheet_id_or_url = spreadsheet_id_or_url
            self.worksheet_name_or_index = worksheet_name_or_index
            self.gc = self.authenticate_and_authorize()
            self.worksheet = self.open_google_sheets()

    def authenticate_and_authorize(self):
        credentials, _ = default()
        try:
            gc = gspread.authorize(credentials)
        except RefreshError:
            credentials.refresh(Request())
            gc = gspread.authorize(credentials)
        return gc

    def open_google_sheets(self):
        if self.spreadsheet_id_or_url.startswith("https://docs.google.com/spreadsheets/d/"):
            spreadsheet = self.gc.open_by_url(self.spreadsheet_id_or_url)
        else:
            spreadsheet = self.gc.open_by_key(self.spreadsheet_id_or_url)
        worksheet = spreadsheet.worksheet(self.worksheet_name_or_index)
        return worksheet

    def get(self, range):
        # Fetch all values from a range of cells.
        range_values = self.worksheet.get(range)
        return range_values

# Execution

In [ ]:
# @title Run {display-mode: "form"}

from google.colab import userdata
import yaml
from datetime import date
from IPython.display import clear_output

# Value of the _U cookie;
# Leave this empty if you are using the bing_cookies_yaml secret.
bing_cookie = ""  # @param {type:"string"}

# Step 1: Create folder 'bing_image_creator_dalle3_batch'
folder_name = 'bing_image_creator_dalle3_batch'
save_folder = gdrive_client.create_folder(folder_name)

# Step 2: Configure class GoogleSheets
spreadsheet_id_or_url = "your_spreadsheet_id_or_url_here"  # @param {type:"string"}
worksheet_name_or_index = "Sheet1"  # @param {type:"string"}
range = "A2:A"  # @param {type:"string"}

gsheets_client = GoogleSheets(spreadsheet_id_or_url, worksheet_name_or_index)
range_values = gsheets_client.get(range)

# Step 3: Loop through the prompts and generate them
for i, value in enumerate(range_values, 1):
    if len(value) == 0:
        print_colored_text(f"Prompt {1}: Cell is empty, skipping...", "red")
        continue  # Skip the loop iteration if the value is empty
    prompt = value[0].strip()
    if prompt:
        # Handle cookies
        if bing_cookie.strip():  # Check if bing_cookie is not empty
            bing_cookies = [bing_cookie]
        else:
            bing_cookies_yaml = userdata.get('bing_cookies_yaml')
            parsed_data = yaml.safe_load(bing_cookies_yaml)  # Parse the YAML content
            bing_cookies = [values for values in parsed_data.values()]  # Get all values as a flat list
        # Configure class BingImageCreator
        num_cookies = len(bing_cookies)  # Calculate the number of cookies in the 'bing_cookies' list
        current_date = date.today().strftime("%Y-%m-%d")  # Construct the current date in the "YYYY-MM-DD" format
        save_folder_with_date = os.path.join(save_folder, current_date) # Append the current date to the save_folder
        os.makedirs(save_folder_with_date, exist_ok=True)  # Create the folder if it doesn't exist
        cookie_index = (i - 1) % num_cookies # Calculate the index to access the bing_cookies list
        print_colored_text(f"Prompt {i}: {prompt}", "black")
        print("Folder path:", save_folder_with_date)
        image_creator = BingImageCreator(prompt,
                                        save_folder=save_folder_with_date,
                                        cookie_value=bing_cookies[cookie_index])
        image_creator.run()
        print()
    # Clear the output every 5 generations
    if i % 5 == 0:
        clear_output(wait=True)

Prompt 1: origami style pikachu . paper art, pleated paper, folded, origami art, pleats, cut and fold, centered composition
Folder path: /content/drive/MyDrive/bing_image_creator_dalle3_batch/2024-02-18
time_to_wait_seconds: 30
Image saved to "/content/drive/MyDrive/bing_image_creator_dalle3_batch/2024-02-18/origami style pikachu . paper art_ pleated paper_ folded_ origami art_ pleats_ cut and fold_ centered composition. Image 1 of 4.jpg".
Image saved to "/content/drive/MyDrive/bing_image_creator_dalle3_batch/2024-02-18/origami style pikachu . paper art_ pleated paper_ folded_ origami art_ pleats_ cut and fold_ centered composition. Image 2 of 4.jpg".
Image saved to "/content/drive/MyDrive/bing_image_creator_dalle3_batch/2024-02-18/origami style pikachu . paper art_ pleated paper_ folded_ origami art_ pleats_ cut and fold_ centered composition. Image 3 of 4.jpg".
Image saved to "/content/drive/MyDrive/bing_image_creator_dalle3_batch/2024-02-18/origami style pikachu . paper art_ pleated